In [1]:
import pandas as pd
import re
import numpy as np

In [2]:
census_df = pd.read_csv('co-est2017-alldata.csv', encoding='latin1')
census_df.head()

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RDOMESTICMIG2015,RDOMESTICMIG2016,RDOMESTICMIG2017,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015,RNETMIG2016,RNETMIG2017
0,40,3,6,1,0,Alabama,Alabama,4779736,4780135,4785579,...,-0.317205,-0.404473,0.788882,0.450741,0.939393,1.364296,0.694271,0.678575,0.558931,1.708218
1,50,3,6,1,1,Alabama,Autauga County,54571,54571,54750,...,-1.950739,4.831269,1.047102,5.911832,-6.102101,-4.050282,2.099325,-1.659040,5.103709,1.317904
2,50,3,6,1,3,Alabama,Baldwin County,182265,182265,183110,...,17.047872,20.493601,22.383175,16.285940,17.196786,22.615285,20.380904,17.903749,21.317244,23.163873
3,50,3,6,1,5,Alabama,Barbour County,27457,27457,27332,...,-16.222436,-18.755525,-19.042395,0.256021,-6.822433,-8.018920,-5.549762,-16.411069,-18.947692,-19.159940
4,50,3,6,1,7,Alabama,Bibb County,22915,22919,22872,...,0.931388,-1.416117,-0.882983,-5.041980,-4.096646,-5.890038,1.243450,1.818424,-0.531044,0.000000


In [3]:
def b0():
    state_df = census_df[census_df['SUMLEV'] == 40]
    return (state_df.iloc[0]['REGION'], state_df.iloc[0]['DIVISION'])

In [4]:
def test_b0():
    region, division = b0()
    if (region, division) == (3,6):
        return True
    return False

test_b0()

True

In [5]:
# create a new dataframe with the same values
co_df = pd.concat([census_df])
# dropping the column which are asked to delete
co_df = co_df.drop('STATE', axis = 1)
co_df = co_df.drop('SUMLEV', axis = 1)
co_df = co_df.drop('COUNTY', axis = 1)
# edit the names of the columns
co_df = co_df.rename(columns={'STNAME':'STATE','CTYNAME':'COUNTY'})
co_df.head()

,REGION,DIVISION,STATE,COUNTY,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,POPESTIMATE2011,POPESTIMATE2012,POPESTIMATE2013,...,RDOMESTICMIG2015,RDOMESTICMIG2016,RDOMESTICMIG2017,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015,RNETMIG2016,RNETMIG2017
0,3,6,Alabama,Alabama,4779736,4780135,4785579,4798649,4813946,4827660,...,-0.317205,-0.404473,0.788882,0.450741,0.939393,1.364296,0.694271,0.678575,0.558931,1.708218
1,3,6,Alabama,Autauga County,54571,54571,54750,55199,54927,54695,...,-1.950739,4.831269,1.047102,5.911832,-6.102101,-4.050282,2.099325,-1.659040,5.103709,1.317904
2,3,6,Alabama,Baldwin County,182265,182265,183110,186534,190048,194736,...,17.047872,20.493601,22.383175,16.285940,17.196786,22.615285,20.380904,17.903749,21.317244,23.163873
3,3,6,Alabama,Barbour County,27457,27457,27332,27351,27175,26947,...,-16.222436,-18.755525,-19.042395,0.256021,-6.822433,-8.018920,-5.549762,-16.411069,-18.947692,-19.159940
4,3,6,Alabama,Bibb County,22915,22919,22872,22745,22658,22503,...,0.931388,-1.416117,-0.882983,-5.041980,-4.096646,-5.890038,1.243450,1.818424,-0.531044,0.000000


In [6]:
def b1():
    # removing the state data and appending the name
    state = co_df.loc[co_df.STATE == co_df.COUNTY]['STATE']
    # empting all the list which are found
    state = state.map({all: None})
    # adding a common value 
    state = state.map({None:'District of Columbia'})
    state = state.unique()
    # changing the values of the DIVISION column with the given names 
    co_df['DIVISION'] = co_df['DIVISION'].map({ 1:'New England', 2:'Middle Atlantic', 3:'East North Central', 4:'West North Central', 5:'South Atlantic', 6:'East South Central', 7:'West South Central', 8:'Mountain', 9:'Pacific'})
    div = co_df['DIVISION'].unique()
    # changing the values of the REGION column with the given names 
    co_df['REGION'] = co_df['REGION'].map({1:'Northeast', 2:'Midwest', 3:'South', 4:'West'})
    region = co_df['REGION'].unique()
    return (list(state), list(div), list(region))

In [7]:
b1()

(['District of Columbia'],
 ['East South Central',
  'Pacific',
  'Mountain',
  'West South Central',
  'New England',
  'South Atlantic',
  'East North Central',
  'West North Central',
  'Middle Atlantic'],
 ['South', 'West', 'Northeast', 'Midwest'])

In [22]:
# Get all the county in New England and Middle Atlantic starting with P
def b2():
    # get the data from the co_df which have the required division
    data_frame1 = co_df[co_df.DIVISION=='New England']
    data_frame2 = co_df[co_df.DIVISION=='Middle Atlantic']
    final_frame = [data_frame1,data_frame2]
   
    data_frame3 = pd.concat(final_frame)
    # get all the county which starts with P
    data_frame3['no1'] = data_frame3['COUNTY'].str.contains('P')
    # get all the state which starts with P
    data_frame3['no2'] = data_frame3['STATE'].str.contains('P')
    # comparing net migrations
    data_frame3['no3'] = np.where((data_frame3['RNETMIG2016']>=data_frame3['RNETMIG2017']),True,False)
    data = data_frame3[(data_frame3['no1']==True)&(data_frame3['no2']==True)&(data_frame3['no3']==True)]
    return(data[['COUNTY','STATE']])

In [23]:
b2()

,COUNTY,STATE
2335,Potter County,Pennsylvania


In [20]:
#get the five countries which have the heighest birth rates from 1 July 2016 to 30 June 2017
def b3():
    alpha = pd.DataFrame()
    alpha['RBIRTH2017'] = co_df['RBIRTH2017']
    alpha['COUNTY'] = co_df['COUNTY']
# finding the five countries with highest birth rates
    return(alpha.nlargest(5,'RBIRTH2017'))

In [21]:
b3()

,RBIRTH2017,COUNTY
935,29.451370,Geary County
83,28.478885,Kusilvak Census Area
596,28.028028,Madison County
2971,27.460145,Fairfax city
2463,26.582027,Todd County


In [12]:
# getting the no of unique county 
def b4():
# select region and division
    unique = co_df[co_df.REGION=='West']
    unique = co_df[co_df.DIVISION=='Mountain']
# get a list with unique values of country
    unique1 = unique['COUNTY'].nunique()
    return(unique1)

In [13]:
b4()

247

In [14]:
def b5():
    # get the five county with heighest frequency
    list_of_frequencies = co_df['COUNTY'].value_counts()
    return(list_of_frequencies[:5])

In [15]:
b5()

Washington County    30
Jefferson County     25
Franklin County      24
Jackson County       23
Lincoln County       23
Name: COUNTY, dtype: int64

In [18]:
# find four states with heighest international migrations
def b6():
    heighest = pd.DataFrame()
    # get the unique values of all the state
    heighest['STATE'] = co_df['STATE'].unique()
    # get the sum of net international migrations for four years
    co_df['ADD']= co_df[['RINTERNATIONALMIG2014','RINTERNATIONALMIG2015','RINTERNATIONALMIG2016','RINTERNATIONALMIG2017']].sum(axis=1) 
    # get the three states with heighest international migrations
    for i in range (len(heighest['STATE'])):
        heighest.loc[[i],'FINAL'] = co_df.loc[co_df['STATE'] == heighest['STATE'][i] ,'ADD'].sum()
    return(heighest.nlargest(3,'FINAL'))

In [19]:
b6()

,STATE,FINAL
43,Texas,2012.093603
46,Virginia,973.865968
9,Florida,761.918478
